In [28]:
from scapy.all import *
from collections import deque
import pandas as pd

## Bienvenidxs al taller de DNS.

A lo largo del taller van a poder probar los diferentes comandos que nos provee la librería Scapy para realizar consultas DNS. El objetivo es que al finalizar el taller tengan el código necesario para poder hacer los experimentos y el informe.

El proceso de resolución de nombres consiste en sucesivas consultas y respuestas por parte de todos los servidores DNS involucrados. Las consultas suelen ser recursivas cuando las PC quieren resolver un nombre y le preguntan al Resolver local y suelen ser iterativas cuando los Resolvers le pasan las consultas a los servidores Autoritativos responsables de cada zona. Por esa razón, en una consulta determinada, puede haber subconsultas recursivas e iterativas. Además de los servidores autoritativos de cada zona, el sistema DNS no podría funcionar si no existieran
servidores por encima de toda la jerarquía de zonas que funcionen como punto de partida para comenzar las
consultas iterativas. Estos servidores se llaman Root Name Servers y tienen direcciones IP asignadas fijas,
que nunca cambian de manera que no haga falta hacer una consulta DNS para resolverlos porque sino no se
podría empezar. Estos servidores y sus direcciones IP están listados en la siguiente tabla:

Nombre del Servidor|Direcciones IP (IPv4, IPv6)|Entidad propietaria
:------------------|:--------------------------|:------------------
a.root-servers.net|198.41.0.4, 2001:503:ba3e::2:30|Verisign, Inc.
b.root-servers.net|199.9.14.201, 2001:500:200::b|University of Southern California
c.root-servers.net|192.33.4.12, 2001:500:2::c|Cogent Communications
d.root-servers.net|199.7.91.13, 2001:500:2d::d|University of Maryland
e.root-servers.net|192.203.230.10, 2001:500:a8::e|NASA (Ames Research Center)
f.root-servers.net|192.5.5.241, 2001:500:2f::f|Internet Systems Consortium, Inc.
g.root-servers.net|192.112.36.4, 2001:500:12::d0d|US Department of Defense (NIC)
h.root-servers.net|198.97.190.53, 2001:500:1::53|US Army (Research Lab)
i.root-servers.net|192.36.148.17, 2001:7fe::53|Netnod
j.root-servers.net|192.58.128.30, 2001:503:c27::2:30|Verisign, Inc.
k.root-servers.net|193.0.14.129, 2001:7fd::1|RIPE NCC
l.root-servers.net|199.7.83.42, 2001:500:9f::42|ICANN
m.root-servers.net|202.12.27.33, 2001:dc3::35|WIDE Project

### Utilizando Scapy consultar por el registro A de www.dc.uba.ar

In [17]:
def dnsRequest(domainName, dnsServer, requestType):
    dns = DNS(rd=1,qd=DNSQR(qname=domainName, qtype=requestType))
    udp = UDP(sport=RandShort(), dport=53)
    try:
        ip = IP(dst=dnsServer)
    except:
        ip = IPv6(dst=dnsServer)

    answer = sr1( ip / udp / dns , verbose=0, timeout=10)

    if answer is not None and answer.haslayer(DNS) and answer[DNS].qd.qtype == requestType:
        
        if answer[DNS].ancount == 0:
            nextSteps = set()
            for i in range( answer[DNS].arcount):
                nextSteps.add(answer[DNS].ar[i].rdata)
            
            return "ANSWER BUT NO FINISHED", nextSteps
        
        else:
            return "FINISHED", answer[DNS]
    else:
        return "NO ANSWER", None

In [18]:
def dig(url,root,requestType):
    visited = set()
    to_Visit = deque()
    to_Visit.append(root)
    
    height = 0
    totalAttempts = 0
    failedAttempts = 0
    
    while len(to_Visit) != 0:
        nextIP = to_Visit.pop()
        height += 1
        totalAttempts += 1
        
        visited.add(nextIP)
        #print(nextIP)
        
        finished, result = dnsRequest(url,nextIP,requestType)
        if (finished == "FINISHED"):
            return result, height, failedAttempts/totalAttempts
        elif (finished == "ANSWER BUT NO FINISHED"):
            
            for ip in result: 
                if ip not in visited:
                    to_Visit.append(ip)
                    
        else:
            height -= 1
            failedAttempts += 1
    raise Exception("No obtuvo respuesta")

In [23]:
def getIP (url):
    root = "198.41.0.4"
    requestTypeA = 1
    
    a_result, _, _ = dig(url,root,requestTypeA)
    
    for i in range( a_result.ancount):
        if(a_result.an[i].type == requestTypeA):
            return a_result.an[i].rdata

In [36]:
# 1 -> A, 15 -> MX
target_url = "dc.uba.ar"
requestTypeMX = 15
requestTypeA = 1
root = "198.41.0.4"
mx_result, height, failureProportion = dig(target_url,root,requestTypeMX)

Exception: No obtuvo respuesta

In [25]:
webServerIP = getIP(target_url)

mailServerIPs = {}
for i in range( mx_result.ancount):
    if(mx_result.an[i].type == requestTypeMX):
        mail_server_name = mx_result.an[i].exchange.decode("utf-8")
        mailServerIPs[mail_server_name] = getIP(mail_server_name)

In [34]:
mailServerIPsPath = "./Resultados/" + target_url + "-MailServerIPs.csv"
resultPath = "./Resultados/" + target_url + "-Results.csv"

df1 = pd.DataFrame([{"Domain": target_url, "Height": height, "Failure proportion": failureProportion, "Web Server IP": webServerIP, "Mail Server IPs Path":mailServerIPsPath}])
df2 =  pd.DataFrame([{"Mail Server": mailServer, "IP": ip} for mailServer, ip in mailServerIPs.items()])

df1.to_csv(resultPath, index = False)
df2.to_csv(mailServerIPsPath, index = False)

## Ejercicio:
Adaptar el código anterior de manera que, a través de sucesivas consultas iterativas se obtenga el registro
MX de un dominio dado. Para esto, tener en cuenta que en cada consulta DNS puede tener 3 tipos de respuestas: 
1. nos devuelven los servidores DNS a los cuales seguir preguntando
2. nos devuelven la respuesta a la consulta que estamos haciendo
3. nos devuelven el registro SOA de la zona indicando que el registro solicitado no forma parte de la base de datos de nombres de la zona.